In [117]:
import pandas as pd
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string
from transformers import BertTokenizer, TFBertForSequenceClassification


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Assistant\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Assistant\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Assistant\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Assistant\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [104]:
df_train = pd.read_csv("Resources/twitter_training.csv", header=None)
df_train.head()

,0,1,2,3
0,2401,Borderlands,Positive,im getting on borderlands and i will murder yo...
1,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
2,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
3,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
4,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...


In [105]:
df_test = pd.read_csv("Resources/twitter_validation.csv",header=None)
df_test.head()

,0,1,2,3
0,3364,Facebook,Irrelevant,I mentioned on Facebook that I was struggling ...
1,352,Amazon,Neutral,BBC News - Amazon boss Jeff Bezos rejects clai...
2,8312,Microsoft,Negative,@Microsoft Why do I pay for WORD when it funct...
3,4371,CS-GO,Negative,"CSGO matchmaking is so full of closet hacking,..."
4,4433,Google,Neutral,Now the President is slapping Americans in the...


In [106]:
df_train = df_train.drop([0,1],axis=1)

In [107]:
df_train = df_train.rename(columns={3:'text'})
df_train = df_train.rename(columns={2:'sentiment'})

In [108]:
df_train['text'].isna().sum()

686

In [109]:
df_train.dropna(subset=['text'],inplace=True)

In [110]:
print('text n/a: ' + str(df_train['text'].isna().sum()))
print('sentiment n/a: ' + str(df_train['sentiment'].isna().sum()))

text n/a: 0
sentiment n/a: 0


In [111]:
df_test = df_test.drop([0,1],axis=1)
df_test = df_test.rename(columns={3:'text'})
df_test = df_test.rename(columns={2:'sentiment'})

In [112]:
print('text n/a: ' + str(df_test['text'].isna().sum()))
print('sentiment n/a: ' + str(df_test['sentiment'].isna().sum()))

text n/a: 0
sentiment n/a: 0


In [113]:
def preprocess_text(text):
    # Lowercase the text
    text = text.lower()
    
    # Remove punctuation
    text = ''.join([char for char in text if char not in string.punctuation])
    
    # Tokenization (split the text into words)
    tokens = nltk.word_tokenize(text)
    
    # Remove stop words
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    
    # Lemmatization (reduce words to their base form)
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    
    # Join the tokens back into a single string
    text = ' '.join(tokens)
    
    return text

# Apply the text pre-processing function to the '3' column
df_train['text'] = df_train['text'].apply(preprocess_text)


In [114]:
df_test['text'] = df_test['text'].apply(preprocess_text)

In [115]:
df_train = pd.get_dummies(df_train, columns=['sentiment'], prefix='sentiment')

In [116]:
df_train.head()

,text,sentiment_Irrelevant,sentiment_Negative,sentiment_Neutral,sentiment_Positive
0,"[im, getting, borderland, murder]",0,0,0,1
1,"[coming, border, kill]",0,0,0,1
2,"[im, getting, borderland, kill]",0,0,0,1
3,"[im, coming, borderland, murder]",0,0,0,1
4,"[im, getting, borderland, 2, murder]",0,0,0,1


In [118]:
model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = TFBertForSequenceClassification.from_pretrained(model_name)

c:\Users\Assistant\anaconda3\envs\dev\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Assistant\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


ImportError: 
TFBertForSequenceClassification requires the TensorFlow library but it was not found in your environment.
However, we were able to find a PyTorch installation. PyTorch classes do not begin
with "TF", but are otherwise identically named to our TF classes.
If you want to use PyTorch, please use those classes instead!

If you really do want to use TensorFlow, please follow the instructions on the
installation page https://www.tensorflow.org/install that match your environment.
